In [106]:
import sys
!{sys.executable} -m pip install airstack
!{sys.executable} -m pip install numpy


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [7]:
import asyncio
import json
from airstack.execute_query import AirstackClient
import csv


In [8]:
api_client = AirstackClient(api_key='9deb75ae44b24f4ca9eecfbc37492edd')

In [9]:
query =  """
query POAPEventHolders {
  Poaps(
    input: {filter: {eventId: {_in: ["661", "49462", "58062", "65219"]}}, blockchain: ALL, limit: 200}
  ) {
    Poap {
      eventId
      owner {
        identity
        primaryDomain {
          name
        }
      }
    }
  }
}
  """
  
variables = {
  "eventId": "103093"
}

In [10]:
execute_query_client = api_client.create_execute_query_object(query=query, variables=variables)
query_response = await execute_query_client.execute_paginated_query()

print("error", query_response.error)
print("data ", json.dumps(query_response.data))

error None
data  {"Poaps": {"Poap": [{"eventId": "58062", "owner": {"identity": "0x7a7560269758b80289eefeef8106571198c40f6d", "primaryDomain": {"name": "woodenship.eth"}}}, {"eventId": "49462", "owner": {"identity": "0x5f4da4b6118142e93f9903d3941fa3d2928219ef", "primaryDomain": null}}, {"eventId": "49462", "owner": {"identity": "0xaf0f557a9a7e84bdd02adbc14962df2789d95d4e", "primaryDomain": null}}, {"eventId": "65219", "owner": {"identity": "0x53b77b9a06dd7e11aee247957784d76484fff540", "primaryDomain": {"name": "xali.eth"}}}, {"eventId": "49462", "owner": {"identity": "0xb400486a30ae70b7293d4587ada484d88418d751", "primaryDomain": null}}, {"eventId": "65219", "owner": {"identity": "0xcb9e0c8a8945986db82b02eab89690a08e01dc04", "primaryDomain": null}}, {"eventId": "49462", "owner": {"identity": "0x7e0244682f26a142e06c5de61da33b9e93067936", "primaryDomain": null}}, {"eventId": "49462", "owner": {"identity": "0xd9e8fb1c5f74e56e1f90d0bf0d5543891dc9283d", "primaryDomain": {"name": "skybfx95.et

In [11]:
if query_response.has_next_page: 
    query_response = await query_response.get_next_page
    for keys in query_response.data:
        print()
    print("page 2 data ", json.dumps(query_response.data))
    
if query_response.has_prev_page: 
    query_response = await query_response.get_prev_page
    print("page 1 data ", json.dumps(query_response.data))


page 2 data  {"Poaps": {"Poap": [{"eventId": "65219", "owner": {"identity": "0x0419395ef65947b74ad9ccd1a9753251e72e411b", "primaryDomain": {"name": "helenesp.eth"}}}, {"eventId": "65219", "owner": {"identity": "0xe030fec019fe9cbab01eab3106eb9ce32851e64d", "primaryDomain": null}}, {"eventId": "65219", "owner": {"identity": "0x367e9c3ac0e28c1536d5503de72bfea24fa4ed8a", "primaryDomain": {"name": "onurofficial.eth"}}}, {"eventId": "65219", "owner": {"identity": "0xf6b6f07862a02c85628b3a9688beae07fea9c863", "primaryDomain": {"name": "poap.eth"}}}, {"eventId": "49462", "owner": {"identity": "0xd54f13ef6775aec76c172b00650a5ac45ae44675", "primaryDomain": {"name": "filou.eth"}}}, {"eventId": "49462", "owner": {"identity": "0x58a8a0c811c21d7a87a27e44df84336208af8e80", "primaryDomain": {"name": "nmoreau.eth"}}}, {"eventId": "661", "owner": {"identity": "0x63101058556d9674b1bd55d408a3dc4113bc65ac", "primaryDomain": {"name": "brianlester.eth"}}}, {"eventId": "49462", "owner": {"identity": "0x3dbf1

In [14]:
async def execute_with_pagination(api_client, query, variables):
    execute_query_client = api_client.create_execute_query_object(query=query, variables=variables)
    query_response = await execute_query_client.execute_paginated_query()
    
    while query_response.has_next_page:
            yield query_response
            query_response = await query_response.get_next_page
    yield query_response

In [15]:
responses =  execute_with_pagination(api_client=api_client,query=query, variables=variables)
output = []
async for response in responses:
    print(response.data)
    if response.data == None: 
        continue
    for value in response.data['Poaps']['Poap']:
        primaryDomain = value['owner']['primaryDomain']
        output.append({
            "eventId": value['eventId'],
            "identity": value['owner']['identity'],
             "primaryENS" :primaryDomain['name'] if  primaryDomain != None  else  ""
        })
        
    #print(response.data)
    #for record in response.Poaps.poap:

csv_file = 'output.csv'

# Write JSON data to CSV
with open(csv_file, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=output[0].keys())
    writer.writeheader()
    writer.writerows(output)

print(f"CSV file '{csv_file}' created successfully.")
   

{'Poaps': {'Poap': [{'eventId': '58062', 'owner': {'identity': '0x7a7560269758b80289eefeef8106571198c40f6d', 'primaryDomain': {'name': 'woodenship.eth'}}}, {'eventId': '49462', 'owner': {'identity': '0x5f4da4b6118142e93f9903d3941fa3d2928219ef', 'primaryDomain': None}}, {'eventId': '49462', 'owner': {'identity': '0xaf0f557a9a7e84bdd02adbc14962df2789d95d4e', 'primaryDomain': None}}, {'eventId': '65219', 'owner': {'identity': '0x53b77b9a06dd7e11aee247957784d76484fff540', 'primaryDomain': {'name': 'xali.eth'}}}, {'eventId': '49462', 'owner': {'identity': '0xb400486a30ae70b7293d4587ada484d88418d751', 'primaryDomain': None}}, {'eventId': '65219', 'owner': {'identity': '0xcb9e0c8a8945986db82b02eab89690a08e01dc04', 'primaryDomain': None}}, {'eventId': '49462', 'owner': {'identity': '0x7e0244682f26a142e06c5de61da33b9e93067936', 'primaryDomain': None}}, {'eventId': '49462', 'owner': {'identity': '0xd9e8fb1c5f74e56e1f90d0bf0d5543891dc9283d', 'primaryDomain': {'name': 'skybfx95.eth'}}}, {'eventId'